In [ ]:
import pandas as pd
import requests

#API_KEY'i bu linkteki bilgileri doldurup elde edebilirsiniz.
#https://bonobo.capi.gutools.co.uk/register/developer

API_KEY = "784462b2-f5e2-43c7-b3cf-f41b613e4122"

DATE = "2023-02-01"
Marcets = ["Apple", "Amazon", "Tesla", "Google", "Microsoft"]
url = f"https://content.guardianapis.com/search?q={Marcets}&from-date={DATE}&api-key={API_KEY}&page=1"

In [ ]:
info=[]

def json(url1):
    response=requests.get(url1)
    if response.status_code == 200:
        x=response.json()
        info.append(x)
    else:
        return False
    return True

for Marcet in Marcets:
    pages = 1
    while True:
        url = f"https://content.guardianapis.com/search?q={Marcet}&from-date={DATE}&api-key={API_KEY}&page={pages}"
        if not json(url):
            break
        pages += 1



KeyboardInterrupt: ignored

In [ ]:
finallist = []

for data in info:
    try:
        for result in data['response']['results']:
            value = dict(
                webtitle=result['webTitle'],
                publisheddate=result['webPublicationDate'],
            )
            finallist.append(value)
    except IndexError:
        print("done")

In [ ]:
datanew=pd.DataFrame(finallist)
datanew

,webtitle,publisheddate
0,‘Ultra-rare’ pair of Apple trainers on sale fo...,2023-07-25T17:40:39Z
1,Dozens injured as apple-sized hailstones hit C...,2023-06-22T12:56:46Z
2,Apple cider vinegar: the ultimate panacea – or...,2023-06-05T09:00:29Z
3,First-generation Apple iPhone sells at auction...,2023-07-17T17:11:52Z
4,Sausage traybake and eve’s pudding: Nancy Birt...,2023-05-22T12:00:33Z
...,...,...
2472,Jimmy Kimmel on Trump in East Palestine: ‘A tr...,2023-02-23T16:00:11Z
2473,Wendy’s to test AI chatbot that takes your dri...,2023-05-10T15:59:01Z
2474,ChatGPT is making up fake Guardian articles. H...,2023-04-06T07:00:16Z
2475,GCHQ warns of fresh threat from Chinese state-...,2023-05-25T15:34:45Z


In [ ]:
def classify_news(title):
    # Bazı haberler, konu ile alakalı olmayabiliyor,
    # 1. Bu kod bloğunu çalıştırmadan devam edin
    # 2. Basit bir çözüm olarak buraya hangi uygun anahtar kelimeler gelmeli, aşağıdakiler anahtar kelimeler deneyin ve onlar dışında kendiz oluşturup deneyin
    # 3. Başka bir çözüm olarak ilgili Type'ları filtreleyip devam edebilirsiniz.
    # 4. Bunun dışında nasıl bir çözüm getirmeliyiz?

    stock_keywords = ['stock', 'shares', 'market', 'trade', 'ipo', 'invest', 'equity',
                      'dividend', 'nasdaq', 'dow jones', 'sp500', 'wall street', 'portfolio', 'Tesla', 'Elon', 'Musk']


    # if any keyword or company name is in the title, classify as 'stock news'
    for keyword in stock_keywords:
        if keyword.lower() in title.lower():
            return 'True'

    # if no keywords or company names are found, classify as 'other news'
    return 'False'

datanew['webtitlerevelant'] = datanew['webtitle'].apply(classify_news)
datanew = datanew[datanew['webtitlerevelant'] != 'False']
print(datanew['webtitlerevelant'].value_counts())

True    144
Name: webtitlerevelant, dtype: int64


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwordsList = stopwords.words('english')

def clean_text(text):

  text = text.lower()

  punctions = ".,:;?!/\"\'`*(){}[]%+^#&‘’|–$€₺"

  for punc in punctions:
    text = text.replace(punc, "")

  text = text.replace("-", " ")
  text = text.replace("_", " ")
  text = text.replace("  ", " ")

  # Step 3: Remove numbers
  text = re.sub(r'\d+', '', text)

  # Step 4: Remove URLs
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

  # Step 5: Tokenization
  wordTokens = nltk.word_tokenize(text)

  # Step 6: Remove Stop Words
  filteredText = [word for word in wordTokens if word not in stopwordsList]

  # Step 7: Lemmatization (running, runner, ran, runs) -> (run, runner, run, run)
  lemmatizedWords = [lemmatizer.lemmatize(word) for word in filteredText]

  # Step 8: Combine the seperated words
  finalText = ' '.join(lemmatizedWords)

  return finalText

datanew['clean_webtitle'] = datanew['webtitle'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-19-e220405ac18a>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datanew['clean_webtitle'] = datanew['webtitle'].apply(clean_text)


In [ ]:
!pip install -q transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

# Load FinBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# Function to predict sentiment
def predict_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt")

    # Make prediction
    outputs = model(**inputs)

    # Get sentiment (neutral, positive, negative)
    sentiment = torch.argmax(outputs.logits, dim=1).item()

    # Return sentiment
    sentiment_mapping = {0: "NEUTRAL", 1: "POSITIVE", 2: "NEGATIVE"}
    return sentiment_mapping[sentiment]

# Apply function to DataFrame column
datanew['sentiment'] = datanew['clean_webtitle'].apply(predict_sentiment)
datanew

<ipython-input-30-cfb61cc9a28b>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datanew['sentiment'] = datanew['clean_webtitle'].apply(predict_sentiment)


,webtitle,publisheddate,webtitlerevelant,clean_webtitle,sentiment
5,"What Apple did to Nokia, Tesla is now doing to...",2023-08-05T15:00:45Z,True,apple nokia tesla motor industry john naughton,NEUTRAL
6,Apple revenues fall for third straight quarter...,2023-08-03T21:39:49Z,True,apple revenue fall third straight quarter comp...,NEGATIVE
54,Big tech is trying to weaken new digital marke...,2023-06-21T23:01:52Z,True,big tech trying weaken new digital market regu...,NEGATIVE
59,Crunch time: food co-ops feeling the squeeze a...,2023-07-27T15:00:45Z,True,crunch time food co ops feeling squeeze austra...,NEGATIVE
65,Watermelons hit the sweet spot: Australia’s be...,2023-03-01T14:00:35Z,True,watermelon hit sweet spot australia best value...,POSITIVE
...,...,...,...,...,...
2435,Google AI chatbot Bard sends shares plummeting...,2023-02-09T01:47:43Z,True,google ai chatbot bard sends share plummeting ...,NEGATIVE
2439,Apple revenues fall for third straight quarter...,2023-08-03T21:39:49Z,True,apple revenue fall third straight quarter comp...,NEGATIVE
2440,Elon Musk reportedly planning to launch AI riv...,2023-04-15T11:16:11Z,True,elon musk reportedly planning launch ai rival ...,NEUTRAL
2442,Pushing Buttons: After a decade of PlayStation...,2023-06-14T10:30:01Z,True,pushing button decade playstation dominance ne...,NEUTRAL


In [ ]:
datanew['sentiment'].value_counts()

NEUTRAL     97
NEGATIVE    25
POSITIVE    22
Name: sentiment, dtype: int64

In [ ]:
from transformers import pipeline

# Load the zero-shot classification pipeline with the BART model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Function to predict related company with a threshold
def predict_company_with_threshold(text, threshold=0.7):  # Set the default threshold here
    # Candidate labels for company classification
    candidate_labels = ["Apple", "Amazon", "Tesla", "Google", "Microsoft"]

    # Classify the text and get the classification results
    classification_result = classifier(text, candidate_labels)

    # Filter predictions based on threshold
    filtered_predictions = [
        label for label, score in zip(classification_result['labels'], classification_result['scores'])
        if score >= threshold
    ]

    if filtered_predictions:
        # Return the most likely company among the filtered predictions
        return filtered_predictions[0]
    else:
        return 'Uncertain'

# Apply function to DataFrame column
datanew['predicted_company'] = datanew['clean_webtitle'].apply(predict_company_with_threshold)
datanew


<ipython-input-41-d17c2459c87d>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datanew['predicted_company'] = datanew['clean_webtitle'].apply(predict_company_with_threshold)


,webtitle,publisheddate,webtitlerevelant,clean_webtitle,sentiment,predicted_company
5,"What Apple did to Nokia, Tesla is now doing to...",2023-08-05T15:00:45Z,True,apple nokia tesla motor industry john naughton,NEUTRAL,Uncertain
6,Apple revenues fall for third straight quarter...,2023-08-03T21:39:49Z,True,apple revenue fall third straight quarter comp...,NEGATIVE,Apple
54,Big tech is trying to weaken new digital marke...,2023-06-21T23:01:52Z,True,big tech trying weaken new digital market regu...,NEGATIVE,Uncertain
59,Crunch time: food co-ops feeling the squeeze a...,2023-07-27T15:00:45Z,True,crunch time food co ops feeling squeeze austra...,NEGATIVE,Uncertain
65,Watermelons hit the sweet spot: Australia’s be...,2023-03-01T14:00:35Z,True,watermelon hit sweet spot australia best value...,POSITIVE,Uncertain
...,...,...,...,...,...,...
2435,Google AI chatbot Bard sends shares plummeting...,2023-02-09T01:47:43Z,True,google ai chatbot bard sends share plummeting ...,NEGATIVE,Google
2439,Apple revenues fall for third straight quarter...,2023-08-03T21:39:49Z,True,apple revenue fall third straight quarter comp...,NEGATIVE,Apple
2440,Elon Musk reportedly planning to launch AI riv...,2023-04-15T11:16:11Z,True,elon musk reportedly planning launch ai rival ...,NEUTRAL,Uncertain
2442,Pushing Buttons: After a decade of PlayStation...,2023-06-14T10:30:01Z,True,pushing button decade playstation dominance ne...,NEUTRAL,Uncertain


In [ ]:
datanew

,webtitle,publisheddate,webtitlerevelant,clean_webtitle,sentiment,predicted_company
5,"What Apple did to Nokia, Tesla is now doing to...",2023-08-05T15:00:45Z,True,apple nokia tesla motor industry john naughton,NEUTRAL,Uncertain
6,Apple revenues fall for third straight quarter...,2023-08-03T21:39:49Z,True,apple revenue fall third straight quarter comp...,NEGATIVE,Apple
54,Big tech is trying to weaken new digital marke...,2023-06-21T23:01:52Z,True,big tech trying weaken new digital market regu...,NEGATIVE,Uncertain
59,Crunch time: food co-ops feeling the squeeze a...,2023-07-27T15:00:45Z,True,crunch time food co ops feeling squeeze austra...,NEGATIVE,Uncertain
65,Watermelons hit the sweet spot: Australia’s be...,2023-03-01T14:00:35Z,True,watermelon hit sweet spot australia best value...,POSITIVE,Uncertain
...,...,...,...,...,...,...
2435,Google AI chatbot Bard sends shares plummeting...,2023-02-09T01:47:43Z,True,google ai chatbot bard sends share plummeting ...,NEGATIVE,Google
2439,Apple revenues fall for third straight quarter...,2023-08-03T21:39:49Z,True,apple revenue fall third straight quarter comp...,NEGATIVE,Apple
2440,Elon Musk reportedly planning to launch AI riv...,2023-04-15T11:16:11Z,True,elon musk reportedly planning launch ai rival ...,NEUTRAL,Uncertain
2442,Pushing Buttons: After a decade of PlayStation...,2023-06-14T10:30:01Z,True,pushing button decade playstation dominance ne...,NEUTRAL,Uncertain


In [ ]:
excel_filename = "zeroshot_t0.7_output.xlsx"  # Change the filename as needed
datanew.to_excel(excel_filename, index=False)

print(f"Data saved as {excel_filename}")

Data saved as zeroshot_t0.7_output.xlsx
